## Creating weather location grid table
1. Import existing grid files.
2. Eliminate non-useful grids.
3. Create new dataframe by iterating through grids and weather for that specific grid.

In [0]:
# Imports
import pandas as pd
import numpy as np
import io
from google.colab import files, drive
import zipfile
import shutil

In [3]:
# Mount Google Drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Set root path, will be different for everyone
root_path = '/content/gdrive/My Drive/U - SE 4455 Software Design/Capstone Public Folder/Data/'

In [0]:
required_files = [
  'Alberta Location /Ivan_NarrowedData/AL_location_grid_wooded.csv.zip',
  'Alberta Location /Shima_NarrowedData/AL_location_grid_usefulness.csv.zip',
  'Weather - Daily Summer Weather per Province/AL_daily_weather_2017_2018.csv.zip',
  'Weather - Stations/alberta_grid_system_with_station.csv.zip',
  'Weather - Stations/current_station_inventory.csv.zip',
]

for local_path in required_files:
  file_path = root_path + local_path
  with zipfile.ZipFile(file_path,"r") as zip_ref:
    zip_ref.extractall('.')


In [0]:
# Import all files into pandas dataframes
location_grid_wooded = pd.read_csv('AL_location_grid_wooded.csv')
location_grid_boreal = pd.read_csv('AL_location_grid_usefulness.csv')
daily_weather = pd.read_csv('AL_daily_weather_2017_2018.csv')
location_grid_stations = pd.read_csv('alberta_grid_system_with_station.csv')
weather_stations = pd.read_csv('current_station_inventory.csv')

In [10]:
daily_weather.head()

,Unnamed: 0,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,181,-112.82,49.05,DEL BONITA AGDM,3042045,2017-07-01,2017,7,1,NaN,28.4,NaN,11.0,NaN,19.7,NaN,0.0,NaN,1.7,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,33.0,NaN,41,NaN
1,182,-112.82,49.05,DEL BONITA AGDM,3042045,2017-07-02,2017,7,2,NaN,28.0,NaN,8.4,NaN,18.2,NaN,0.0,NaN,0.2,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,35.0,NaN,32,NaN
2,183,-112.82,49.05,DEL BONITA AGDM,3042045,2017-07-03,2017,7,3,NaN,28.4,NaN,11.7,NaN,20.1,NaN,0.0,NaN,2.1,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,24.0,NaN,50,NaN
3,184,-112.82,49.05,DEL BONITA AGDM,3042045,2017-07-04,2017,7,4,NaN,28.5,NaN,8.4,NaN,18.5,NaN,0.0,NaN,0.5,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,16.0,NaN,41,NaN
4,185,-112.82,49.05,DEL BONITA AGDM,3042045,2017-07-05,2017,7,5,NaN,NaN,M,7.4,E,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,NaN,NaN,NaN,<31,NaN


In [11]:
# Leave only useful grids
print(f'Total grids: {len(location_grid_stations)}')
only_wooded_grid_locations = location_grid_wooded[location_grid_wooded['wooded'] == 1]
useful_grids = pd.merge(location_grid_boreal, only_wooded_grid_locations, on='KEY')['KEY']
print(f'Useful grids: {len(useful_grids)}')

useful_location_grid_stations = pd.merge(useful_grids, location_grid_stations, on='KEY')
print(f'Eliminated: {len(location_grid_stations) - len(useful_location_grid_stations)}')

Total grids: 2296
Useful grids: 1833
Eliminated: 463


In [12]:
# Rename key name
weather_stations.rename(columns={'Name': 'Station Name', 'Station ID': 'STATION ID'}, inplace=True)
weather_stations = weather_stations[['Station Name', 'STATION ID']]

# Get daily weather ready for merging
daily_weather_with_station = pd.merge(weather_stations, daily_weather, on='Station Name')
daily_weather_with_station.head()

,Station Name,STATION ID,Unnamed: 0,Longitude (x),Latitude (y),Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,EGG ISLAND,389,35951,-110.44,58.98,3072246,2017-07-01,2017,7,1,NaN,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,NaN,NaN,M,NaN,M
1,EGG ISLAND,389,35952,-110.44,58.98,3072246,2017-07-02,2017,7,2,NaN,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,NaN,NaN,M,NaN,M
2,EGG ISLAND,389,35953,-110.44,58.98,3072246,2017-07-03,2017,7,3,NaN,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,NaN,NaN,M,NaN,M
3,EGG ISLAND,389,35954,-110.44,58.98,3072246,2017-07-04,2017,7,4,NaN,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,NaN,NaN,M,NaN,M
4,EGG ISLAND,389,35955,-110.44,58.98,3072246,2017-07-05,2017,7,5,NaN,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,NaN,NaN,M,NaN,M


In [13]:
weather_location_grid = pd.merge(useful_location_grid_stations, daily_weather_with_station, on='STATION ID')
weather_location_grid.head()

,KEY,Unnamed: 0_x,STATION ID,Station Name,Unnamed: 0_y,Longitude (x),Latitude (y),Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,49.0|-113.8,0,26850,WATERTON PARK GATE,26461,-113.81,49.13,3056214,2017-07-01,2017,7,1,NaN,27.1,NaN,12.2,NaN,19.7,NaN,0.0,NaN,1.7,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,1.0,NaN,52,NaN
1,49.0|-113.8,0,26850,WATERTON PARK GATE,26462,-113.81,49.13,3056214,2017-07-02,2017,7,2,NaN,28.5,NaN,7.6,NaN,18.1,NaN,0.0,NaN,0.1,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,21.0,NaN,33,NaN
2,49.0|-113.8,0,26850,WATERTON PARK GATE,26463,-113.81,49.13,3056214,2017-07-03,2017,7,3,NaN,27.6,NaN,10.4,NaN,19.0,NaN,0.0,NaN,1.0,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,26.0,NaN,57,NaN
3,49.0|-113.8,0,26850,WATERTON PARK GATE,26464,-113.81,49.13,3056214,2017-07-04,2017,7,4,NaN,27.1,NaN,9.4,NaN,18.3,NaN,0.0,NaN,0.3,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,20.0,NaN,48,NaN
4,49.0|-113.8,0,26850,WATERTON PARK GATE,26465,-113.81,49.13,3056214,2017-07-05,2017,7,5,NaN,27.7,NaN,7.0,NaN,17.4,NaN,0.6,NaN,0.0,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,2.0,NaN,37,NaN


In [0]:
# Clean up table
weather_location_grid.rename(columns={'KEY': 'LOCATION KEY'}, inplace=True)
del weather_location_grid['Unnamed: 0_x']
del weather_location_grid['Unnamed: 0_y']
weather_location_grid['KEY'] = weather_location_grid['LOCATION KEY'] + '|' + weather_location_grid['Date/Time']

In [15]:
weather_location_grid.set_index('KEY', inplace=True)
weather_location_grid.head()

,LOCATION KEY,STATION ID,Station Name,Longitude (x),Latitude (y),Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
KEY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
49.0|-113.8|2017-07-01,49.0|-113.8,26850,WATERTON PARK GATE,-113.81,49.13,3056214,2017-07-01,2017,7,1,NaN,27.1,NaN,12.2,NaN,19.7,NaN,0.0,NaN,1.7,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,1.0,NaN,52,NaN
49.0|-113.8|2017-07-02,49.0|-113.8,26850,WATERTON PARK GATE,-113.81,49.13,3056214,2017-07-02,2017,7,2,NaN,28.5,NaN,7.6,NaN,18.1,NaN,0.0,NaN,0.1,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,21.0,NaN,33,NaN
49.0|-113.8|2017-07-03,49.0|-113.8,26850,WATERTON PARK GATE,-113.81,49.13,3056214,2017-07-03,2017,7,3,NaN,27.6,NaN,10.4,NaN,19.0,NaN,0.0,NaN,1.0,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,26.0,NaN,57,NaN
49.0|-113.8|2017-07-04,49.0|-113.8,26850,WATERTON PARK GATE,-113.81,49.13,3056214,2017-07-04,2017,7,4,NaN,27.1,NaN,9.4,NaN,18.3,NaN,0.0,NaN,0.3,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,20.0,NaN,48,NaN
49.0|-113.8|2017-07-05,49.0|-113.8,26850,WATERTON PARK GATE,-113.81,49.13,3056214,2017-07-05,2017,7,5,NaN,27.7,NaN,7.0,NaN,17.4,NaN,0.6,NaN,0.0,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,2.0,NaN,37,NaN


In [0]:
weather_location_grid.to_csv('weather_location_grid.csv.gz', compression='gzip')

In [0]:
# Download the file locally 
files.download('weather_location_grid.csv.gz')

In [21]:
# Save file to google drive
file_path = root_path + 'Alberta Location /AL_weather_location_grid.csv.gz'
shutil.copy('weather_location_grid.csv.gz', file_path)

'/content/gdrive/My Drive/U - SE 4455 Software Design/Capstone Public Folder/Data/Alberta Location /AL_weather_location_grid.csv.gz'